# RunnablePassThrough
chain을 여러 단계로 구성할때 전 단계의 출력값을 바로 다음 단계의 입력값으로 사용해야하는 경우가 있는데

이런 경우에 `RunnablePassThrough`를 사용하면 됨. 일반적으로 `RunnableParallel`이랑 같이 사용됨

# 1. API key 설정

In [3]:
%pip install -qU langchain langchain-openai

import os
from getpass import getpass

#os.environ["OPENAI_API_KEY"] = getpass()

Note: you may need to restart the kernel to use updated packages.


실제 실행시에는 본인 api를 입력해야함

# 2. RunnablePassTrough 구조 살피기

In [2]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed = RunnablePassthrough(),
    modified = lambda x : x["num"] + 1,
)

runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'modified': 2}

- 위 코드에서는 `RunnablePassThrough()`를 통해서 값을 전달받고 `passed` 변수에 저장함.
- 이후 modified에 수정된 값을 저장함.
- modified에는 람다식으로 앞서 passed 된 값에 1을 더해줌.
    - `lambda x` : 여기서 x는 입력값
    - `x["num"] + 1` : x는 딕셔너리 값이므로 num이라는 key를 가진 value에 1을 더해줌

# 3. Retrieval Example

In [8]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

retrieval_chain = (
    {"context" : retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

- Retrival_chain은 {"context", "question"}, prompt, model, StrOutputParser()로 구성된다.
- "context"는 vector store에서 검색을 해서 가져오는 문자열
- "question"은 `RunnablePassThrough` 즉,invoke() 안의 문자열이다.
- 이후 "context"와 "question"이 prompt에 formatting 된다.
- 이후 이 prompt로 model에 질의를 하고 
- StrOutputParser()를 통해서 결과를 문자열로 출력한다.

### StrOutputParser()가 없는 경우 아래와 같다.

In [9]:
retrieval_chain = (
    {"context" : retriever, "question": RunnablePassthrough()}
    | prompt
    | model
)
retrieval_chain.invoke("where did harrison work?")

AIMessage(content='Harrison worked at Kensho.', response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 39, 'total_tokens': 46}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-94234e1a-39dd-447f-9ee1-fbb23e98156f-0', usage_metadata={'input_tokens': 39, 'output_tokens': 7, 'total_tokens': 46})